In [ ]:
import pandas as pd
import re
import spacy
import nltk
import pymorphy2
import csv
from pandas import read_csv
pt='/Users/Daniel/Downloads/Log.txt'
#читаем файл
data_log = pd.read_csv(pt, sep='\t', encoding='utf8', parse_dates=['datetime'], dayfirst=True)
# выводим содержимое для предварительной визуальной оценки
data_pandas = pd.DataFrame(data_log)
pd.options.display.max_rows = 100
pd.options.display.max_colwidth=10000
#display(data_pandas)
#видим, что у нас всего 1333503 строк с ззапросами
#выбираем столбец, с которым будем работать
data_request = data_log['request']
#Выделим запросы
data_request = [doc.replace("https://yandex.ru/search/?text=", "") for doc in data_request]
print ("Закончено 1")
data_request = [doc.lower() for doc in data_request] #приводим все к одному регистру
print ("Закончено 2")
data_request = [doc[:doc.find('&')] for doc in data_request] #
print ("Закончено 3")
data_request = [re.sub( r'\b\d+\b', '', doc ) for doc in data_request] #  удаляем  числительные
print ("Закончено 4")
data_request = [x for x in data_request if x != ''] #  удаляем  пустые запросы
#Можно еще удалить url, но я планировал проанализировать их в отдельном списке, только немного не успел, подздно начал :-)

#from nltk.stem import  SnowballStemmer
#stemmer = SnowballStemmer('russian')
#stem=[' '.join ([stemmer.stem(w) for w in txt.split()]) for txt in (data_request)]
#data_normal = (stem)

#Если задачу понимать дословно, и решать быстрои и просто, то в запросе должно быть слово   "смотреть", если пользователь ищет "фильм", не и все, что связано с телевидением, 
# которое смотреть, как правило, нельзя, можно смотреть только телевизор, передачу, фильм, кино, видео. 
#Если понимать "телевидение", как технологию для передачи видеоинформации,то видеоролики, записанные на диски
#и запрещенные к показу в общественном эфире тоже необходимо относить к поставленной задаче, только если их смотрят.
# Исходя их этих рассуждений формируем 2 словаря 
# Первый с основами синонимов слова "смотреть" учитывая основные языки
keywords_1 = ['смотр', 'cvjnh', 'смoтр' , 'смотp','смотр', 'гляд', 'глаз',
              'regard' , 'voir' , 'vision' , 'mater', 'look' , 'see', 'watc', 'behold',
              'view' ,'overlook', 'stare', 'schauen', 'sehen', 'betrachten', 'blicken', 'zusehen',
              'онлайн','jykfqy', 'online']
# Второй, связанный с телевидением
keywords_2 = ['телев', 'тeлев', 'телeв', 'тeлeв', 'ntktd', 
              'dram','720', 'сезон','ctpjy','1080', 'show', 'погоди','зщкт', 'рщrn',
              'sitcom','horror', 'news', 'новос', 'comed', 
              'прик', 'thril', 'телеб', 'драм', 'шоу', 'ужас',
              'трилл', 'комед', 'broadc', 'ferns','трансляц', 'nhfyckzw',
              'tv', 'тв', 'nd' , 'ем','hd', 'лайн', 'online', 'спорт', 'sport', 'футбол', 'aen,jk',
              'бесплат', 'порн', 'porn','кино', 'chan','sputn','сери', 'cthb' 
              'серіа', 'series', 'serie' ,'фильм', 'програм', 'передач', 
              'виде', 'канал' , 'спутников' , 'эфир', 'вещан', 'сигнал', 
              'кадр', 'режисс', 'актер', 'telev', 'kino', 'film', 'program',
              'transmissions', 'vide', 'channel', 'satellites', 'broadcast', 
              'broadcast', 'signal', 'frame', 'regiss', 'actor', 'cinéma', 'film', 
              'plus', 'transmission', 'canal' , 'satellites' , 'émission', 'вещан', 
              'signal', 'image', 'режисс', 'acteur' , 'koste', 'kino', 'film', 'programme', 
              'getriebe', 'form', 'kanal' , 'satelliten' , 'äther', 'вещан', 'signal', 
              'frame','schauspieler', 'телеві', 'кіно', 'фільм', 'програма', 'передач', 
              'вигляді', 'канал' ,'супутників', 'ефір', 'вещан', 'сигнал', 'кадр', 'режисерка', 'актор', 'нвт', 'нтв',
              'стс', 'орт', 'россия24', 'кино', 'тнт', 'дождь', 'домашний', 'канал', 'пятница', 'ije', 'мультфильм',
              'триколор']
i=0
req_key_1 = []
req_key_2 = []
for subscr in data_request:
#Сначала ищем по первому словарю    
    for word in keywords_1:
        if (word in subscr):        
            req_key_1.append(subscr)
            break
print (len(req_key_1))

#Потом по второму           
for subscr2 in req_key_1:
    for word2 in keywords_2:
        if (word2 in subscr2):        
            req_key_2.append(subscr2)
            break
print (len(req_key_2))
print (req_key_2)

print ("Общее число запросов по просмотру {}" .format(len(req_key_1)))
print ("Общее число запросов по просмотру телевидения {}" .format(len(req_key_2)))
f= len(req_key_2)/len(data_pandas)
print ("Доля запросов, связанная с просмотром и телевидения {}" .format (round(f,2)))
# Теперь попробуем выделить темы и значимость каждой, код повторяется, да, противоречит всем правилам, но это из-за недостатка времении

req_key_1 = [doc.replace("https://yandex.ru/search/?text=", "") for doc in data_request]
print ("Закончено 1")
req_key_1 = [doc.lower() for doc in data_request] #приводим все к одному регистру
print ("Закончено 2")
req_key_1 = [doc[:doc.find('&')] for doc in data_request] #
print ("Закончено 3")
req_key_1 = [re.sub( r'\b\d+\b', '', doc ) for doc in data_request] #  удаляем  числительные
print ("Закончено 4")
req_key_1 = [x for x in data_request if x != ''] #  удаляем  пустые запросы

from nltk.stem import  SnowballStemmer
stemmer = SnowballStemmer('russian')
stem=[' '.join ([stemmer.stem(w) for w in txt.split()]) for txt in (req_key_1)]
data_normal_2 = (stem)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.pipeline import make_pipeline

ru_stop_words = set (['и', 'в', 'не', 'что', 'он', 'на', 'я', 'где' 'с', 'со', 'как', 'а',
                      'то', 'все', 'она', 'так', 'его', 'но', 'да','ты', 'к', 'у', 'же',
                      'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от',
                      'меня','еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'нру',
                      'вдруг', 'ли', 'если', 'уже','для', 'про', 'где', 'эт', 'как', 'почему', 
                      'где', 'сколько','а', 'б', 'в', 'г', 'д','е', 'ё', 'ж', 'з', 'и', 'к', 
                      'л', 'м','н', 'о', 'п', 'р', 'с','т', 'у', 'ф', 'х', 'ц', 'ч', 'ш',
                      'щ', 'э','ю', 'я', 'q', 'w', 'e','r', 't', 'y', 'u', 'i', 'o', 'p',
                      'a', 's','d', 'f', 'g', 'h', 'j','k', 'l', 'z', 'x', 'c', 'v', 'b',
                      'n', 'm','0', '1', '2', '3','4', '5', '6', '7', '8', '9'])
my_stop_words = text.ENGLISH_STOP_WORDS.union (ru_stop_words)

#from sklearn.feature_extraction.text import CountVectorizer
#vect = CountVectorizer(min_df=2,stop_words = my_stop_words).fit(data_normal_s)
#X_train= vect.fit_transform (data_normal_s)

tfidf_v = TfidfVectorizer(min_df=2, max_df=0.95, stop_words = my_stop_words)
X = tfidf_v.fit_transform (data_normal_2)

from sklearn.decomposition import LatentDirichletAllocation
print ("Начало построения кластера")
topics = 10 #Попробуем выделить 10 тем
lda = LatentDirichletAllocation(n_components = topics, learning_method="online",
                                max_iter=30, random_state=100, n_jobs=1, total_samples=2000000.0)
document_topics = lda.fit_transform(X)



print ("Закончена кластеризация")

print("lda.components_.shape: {}".format(lda.components_.shape))
print ("Построение вывода 1")

topics = np.array (range(topics))

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]

print ("Построение вывода 2")
feature_names = np.array(tfidf_v.get_feature_names())

print ("Построение вывода 3")
mglearn.tools.print_topics(topics=topics, feature_names=feature_names, 
                           sorting=sorting, topics_per_chunk=5, n_words=15)
print (document_topics)


for j in topics:
    music = np.argsort(document_topics[:,j])[::-1]
    print (document_topics[:,j])
    print ("Тема {}".format(topics[j]))
    print ((music))
    for i in music[:20]:
        print(req_key_1[i])

fig, ax = plt.subplots(1, 2, figsize=(10, 10))
topic_names = ["{:>2} ".format(i) + " ".join(words)
               for i, words in enumerate(feature_names[sorting[:, :2]])]
# two column bar chart:
for col in [0, 1]:
    start = col * 20
    end = (col + 1) * 20
    ax[col].barh(np.arange(20), np.sum(document_topics, axis=0)[start:end])
    ax[col].set_yticks(np.arange(20))
    ax[col].set_yticklabels(topic_names[start:end], ha="left", va="top")
    ax[col].invert_yaxis()
    ax[col].set_xlim(0, 155000)
    yax = ax[col].get_yaxis()
    yax.set_tick_params(pad=130)
plt.tight_layout()   
      





Закончено 1
Закончено 2
Закончено 3
Закончено 4
47572
38239
['футбол  смотреть онлай', 'и всётаки я люблю все серии подряд онлайн бесплатно в хорошем качеств', 'видео разговор девушек про секс.смотреть бесплатн', 'смотреть онлайн матч футбол  прямая трансляци', 'смотреть русское порн', 'онлайн заявка на кредитную карту', 'радио релакс слушать онлайн', 'рома шахтер прямая трансляция смотреть  март', 'любительское двойное проникновение с аналом в жену, свингер-оргии с аналом, невеста, нарезка, онлайн', 'команда б смотреть онлайн все серии в хорошем качестве бесплатно', 'второй канал онлайн прямой эфир россия ', 'смотреть ходячие мертвецы  сезон  сери', 'ветряная река смотреть онлайн', 'смотреть порно самый большой член', 'сериал последний москаль  сезон смотреть онлайн все серии', 'кино онлайн', 'фермер ищет жену  сезон смотреть онлайн', 'карты пасьянсы играть онлайн бесплатно без регистрации', 'смотреть дом  свежие серии бесплатно', 'смотреть фильм онлайн крепость бадабе', 'тихий дон фи

Закончено 1
Закончено 2
Закончено 3
Закончено 4
